In [1]:
import numpy as np

## 卷积学习

In [3]:
x = [6,2]
h = [1,2,5,4]

# x作为卷积内核，以滑动窗口形式覆盖h，默认会将内核倒置
y = np.convolve(x, h)
y

array([ 6, 14, 34, 34,  8])

In [4]:
# “full”是默认的两侧全部填充0，“same”只在左侧填充，而“valid”则不填充。
print(np.convolve(x,h,"full"))  
print(np.convolve(x,h,"same"))  
print(np.convolve(x,h,"valid"))  

[ 6 14 34 34  8]
[ 6 14 34 34]
[14 34 34]


In [5]:
from scipy import signal as sg

 其中g为卷积内核，默认会将内核旋转180°再进行卷积，
 所以full形式中，第一行-2由[[3,2],[1,-1]]矩阵点乘[[0,0],[0,2]]矩阵再求和得到；
 第二个-2由[[3,2],[1,-1]]矩阵点乘[[0,0],[2,4]]得到；其余类似。
 其中“full”为全部填充0，“same”为左侧和上册填充0，而“valid”则全不填充。

In [6]:
I = [[2,4], 
    [11,22]]
g = [[-1, 1],
    [2, 3]]
# 其中g为卷积内核，默认会将内核旋转180°再进行卷积，
# 所以full形式中，第一行-2由[[3,2],[1,-1]]矩阵点乘[[0,0],[0,2]]矩阵再求和得到；
# 第二个-2由[[3,2],[1,-1]]矩阵点乘[[0,0],[2,4]]得到；其余类似。
# 其中“full”为全部填充0，“same”为左侧和上册填充0，而“valid”则全不填充。
print('full type')
print(sg.convolve(I, g, "full"))  
print('same type')
print(sg.convolve(I, g, "same"))  
print('valid type')
print(sg.convolve(I, g, "valid"))  

full type
[[-2 -2  4]
 [-7  3 34]
 [22 77 66]]
same type
[[-2 -2]
 [-7  3]]
valid type
[[3]]


In [8]:
import tensorflow as tf

/Users/hornsey/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
tf.VERSION

'1.7.0'

图层和内核数值都由随机数产生，而随机数后面的参数设置了其形状，具体如下：

input后面[1,3,3,1]表示[批量大小,宽度,高度,通道数目]

filter后面[2,2,1,1]表示[宽度,高度,通道,滤波器数目]

In [10]:
input = tf.Variable(tf.random_normal([1,3,3,1]))
filter = tf.Variable(tf.random_normal([2,2,1,1]))
op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID')

In [11]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    print("Input \n")
    print("{} \n".format(input.eval()))
    print("Filter Kernel \n")
    print("{} \n".format(filter.eval()))
    print("Result Feature Map with VALID position \n")
    result = sess.run(op)
    print(result)

Input 

[[[[-0.7677195 ]
   [-0.32693413]
   [-0.54436994]]

  [[-0.23516177]
   [ 0.5091488 ]
   [ 0.5215869 ]]

  [[-1.1619495 ]
   [ 0.54857546]
   [ 0.8190961 ]]]] 

Filter Kernel 

[[[[ 0.15619908]]

  [[ 1.374759  ]]]


 [[[ 0.25898153]]

  [[-1.0108703 ]]]] 

Result Feature Map with VALID position 

[[[[-1.1449586 ]
   [-1.1948409 ]]

  [[-0.19223735]
   [ 0.1106558 ]]]]


## 测试数据集

### 读取所有数字

In [14]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
 
 
def load_mnist(path, kind="train"):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
 
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        # 'I'表示一个无符号整数，大小为四个字节
        # '>II'表示读取两个无符号整数，即8个字节
        labels = np.fromfile(lbpath, dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols =  struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels
 
 
X_train, y_train = load_mnist("MNIST_data/", kind="train")
X_test, y_test = load_mnist("MNIST_data/", kind="t10k")
 
fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
 
ax = ax.flatten()
for i in range(10):
    img = X_train[y_train == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
 
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'MNIST_data/train-labels.idx1-ubyte'

### 读取某个数字的多张图片

In [15]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
 
 
def load_mnist(path, kind="train"):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
 
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols =  struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels
 
 
X_train, y_train = load_mnist("MNIST_data/", kind="train")
X_test, y_test = load_mnist("MNIST_data/", kind="t10k")
 
fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)
 
ax = ax.flatten()
for i in range(25):
    img = X_train[y_train == 9][i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
 
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'MNIST_data/train-labels.idx1-ubyte'

## 手写数字识别

### 读取数据

In [13]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

sess = tf.InteractiveSession()
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

width = 28 #以像素为单位的图像宽度
height = 28 #以像素为单位的图像高度
flat = width * height # 一个图像中的像素数
class_output = 10 # 问题可能分类的数量

x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

x_image = tf.reshape(x, [-1,28,28,1])  

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 设置卷积层1

1、这里过滤器内核大小是5\*5; 输入通道为1（灰度）；我们需要32个不同的特征映射（这里，32个特征映射表示32个不同的滤波器应用在每个图像上，因此卷积层的输出将是28\*28\*32）.

2、ReLU激励函数的作用是，将covolve1中负数出现的任何地方，用0替换。

3、此处最大池的内核为2\*2，接移动步长为2，即将内核做滑动 窗口作用于图像，且取窗口内最大值。此处达到了降维的效果，最后输出为14\*14\*32的矩阵。

In [ ]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
#ReLU激励函数
h_conv1 = tf.nn.relu(convolve1)
#最大池
conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 设置卷积层2

此处设置完成了第二层卷积。第一到第二层，输入的是[14x14x32]的图像，过滤器是[5x5x32]的内核，我们使用尺寸[5x5x32]的64级的过滤器，即取了64个特征。所以最后通过最大池，第二层输出的是[7x7x64]

In [ ]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #需要64输出的64个偏差
convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2
#ReLU激励函数
h_conv2 = tf.nn.relu(convolve2)
#最大池
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

### 设置第三层完全连接层

In [ ]:
layer2_matrix = tf.reshape(conv2, [-1, 7*7*64])
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) 
fcl=tf.matmul(layer2_matrix, W_fc1) + b_fc1# need 1024 biases for 1024 outputs

h_fc1 = tf.nn.relu(fcl)#ReLU激励函数

### 设置读出层

这是网络“忘记”某些功能的阶段。在小批量的每个培训步骤中，一些设备会随机关机，以免它们与网络互动。也就是说，它的权重不能被更新，也不会影响其他网络节点的学习。这对于非常大的神经网络来防止过拟合非常有用。

In [ ]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1)) #1024个神经元
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10])) 
fc=tf.matmul(layer_drop, W_fc2) + b_fc2
y_CNN= tf.nn.softmax(fc)# 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

### 训练函数

定义损失函数，优化器

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_CNN,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess.run(tf.global_variables_initializer())
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))